# 디스크립터 개요
### 디스크립터 메커니즘
디스크립터를 구현하기 위해서는 최소 두 개의 클래스 필요
1. 클라이언트 클래스: 디스크립터 구현의 기능을 활용할 도메인 모델. 솔루션을 위해 생성한 일반적인 추상화 객체
1. 디스크립터 클래스: 디스크립터 로직의 구현체  

**디스크립터:** 디스크립터 프로토콜을 구현한 클래스의 인스턴스. 이 클래스는 다음 매직 메서드 중에 최소 한 개 이상을 포함해야 함  
- &#95;&#95;get&#95;&#95;
- &#95;&#95;set&#95;&#95;
- &#95;&#95;delete&#95;&#95;
- &#95;&#95;set&#95;name&#95;&#95;

**네이밍 컨벤션**
- ClientClass: 클라이언트 클래스. 클래스 속성으로 디스크립터를 가짐  
- DescriptorClass: 디스크립터 클래스. 디스크립터 프로토콜을 따르는 매직 메서드를 구현해야 함  
- client: ClientClass의 인스턴스. client = ClientClass()
- descriptor: DescriptorClass의 인스턴스. descriptor = DescriptorClass  

**주의할 점:** 이 프로토콜이 동작하려면 디스크립터 객체가 클래스 속성으로 정의되어야 함. 이 객체를 인스턴스 속성으로 생성하면 동작하지 않으므로 init 메서드가 아니라 클래스 본문에 있어야 함 (DescriptorClass() -> X / DescriptorClass -> O)  

디스크립터 프로토콜의 일부만 구현해도 됨.   
디스크립터 클래스 & 디스크립터의 로직을 사용하는 클래스(디스크립터 객체를 멤버로 갖는 클래스)  
ClientClass의 인스턴스에서 descriptor 속성을 호출하면 디스크립터 프로토콜이 사용됨  

일반적인 클래스의 속성 or 프로퍼티에 접근하면 아래와 같은 예상된 결과를 얻을 수 있음

In [2]:
class Attribute:
    value = 42
    
class Client:
    attribute = Attribute()
    
print(Client().attribute)
print(Client().attribute.value)

42


그러나 디스크립터의 경우 다르게 동작.  
클래스 속성을 객체로 선언하면 디스크립터로 인식되고, 클라이언트에서 해당 속성을 호출하면 객체 자체를 반환하는 것이 아니라 &#95;&#95;get&#95;&#95;메서드의 결과를 반환함

In [16]:
import logging

logger = logging.getLogger(__name__)

class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        
        logger.info("Call: %s.__get__(%r, %r)", self.__class__.__name__, instance, owner)
        print(f"Call: {self.__class__.__name__}.__get__({instance}, {owner})")
        return instance
    
class ClientClass:
    descriptor = DescriptorClass()
    
client = ClientClass()
print(client.descriptor)
print(client.descriptor is client)

Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x000001C66E78AAF0>, <class '__main__.ClientClass'>)
Call: DescriptorClass.__get__(<__main__.ClientClass object at 0x000001C66E78AAF0>, <class '__main__.ClientClass'>)
True


이 예제에서는 클라이언트 자체를 그대로 반환했으므로 마지막 비교문장은 True가 됨  
디스크립터를 사용해 &#95;&#95;get&#95;&#95; 메서드 뒤쪽으로 모든 종류의 논리를 추상화할 수 있으며 클라이언트에게 내용을 숨김 채로 모든 유형의 변환을 투명하게 실행할 수 있음

### 디스크립터 프로토콜의 메서드 탐색
디스크립터는 단지 객체이므로 이러한 메서드들은 self를 첫번째 파라미터로 사용함.  
self는 디스크립터 객체 자신을 의미함

#### &#95;&#95;get&#95;&#95;(self, instance, owner)
- instance 파라미터: 디스크립터를 호출한 객체  
- owner 파라미터: 해당 객체의 클래스  
- instance 파라미터는 디스크립터가 행동을 취하려는 객체이고 owner는 인스턴스의 클래스임.
- owner = instance.&#95;&#95;class&#95;&#95; 형태로 instance에서 클래스를 직접 구할 수 있음에도 굳이 owner를 시그니처에 정의한 이유
    - client 인스턴스가 아니라 ClientClass에서 descriptor를 호출하는 특별한 경우에 instance의 값은 None이기 때문에 클래스를 구할 수 없기 때문  
    
아래 예제는 디스크립터가 클래스에서 호출될 때와 인스턴스에서 호출될 때의 차이에 대한 예제. 

In [17]:
class DescriptorClass:
    def __get__(self, instance, owner):
        if instance is None:
            return f"{self.__class__.__name__}.{owner.__name__}"

class ClientClass:
    descriptor = DescriptorClass()
    
# ClientClass에서 직접 호출
print(ClientClass.descriptor)

# 객체에서 호출
print(ClientClass().descriptor)

DescriptorClass.ClientClass
None


일반적으로 owner 파라미터를 사용하려는 경우가 아니라면 인스턴스가 None일 떄는 단순히 디스크립터 자체를 반환.  

#### &#95;&#95;set&#95;&#95;(self, instacne, value)
이 메서드는 디스크립터에 값을 할당하려고 할 때 호출됨  
&#95;&#95;set&#95;&#95;() 메서드를 구현한 디스크립터에 대해서만 활성화 됨.
```python
client.descriptor = "value"
```
위의 코드의 경우 instance 파라미터는 client이고 value는 "value"라는 문자열임  
client.descriptor가 &#95;&#95;set&#95;&#95;() 메서드를 구현하지 않았으면 "value"는 descriptor 자체를 덮어쓰게 됨  


아래 코드는 이 메서드를 사용하여 속성의 유효성을 검사하는 객체를 만드는 코드임.

In [24]:
class Validation:
    def __init__(self, validation_function, error_msg: str):
        self.validation_function = validation_function
        self.error_msg = error_msg
        
    def __call__(self, value):
        if not self.validation_function(value):
            raise ValueError(f"{value!r} {self.error_msg}")
            
class Field:
    def __init__(self, *validations):
        self._name = None
        self.validations = validations
        
    def __set_name__(self, owner, name):
        self._name = name
        
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return instance.__dict__[self._name]
    
    def validate(self, value):
        for validation in self.validations:
            validation(value)
            
    def __set__(self, instance, value):
        self.validate(value)
        instance.__dict__[self._name] = value
        
class ClientClass:
    descriptor = Field(
        Validation(lambda x: isinstance(x, (int, float)), "는 숫자가 아님"), 
        Validation(lambda x: x>= 0 ,"는 0보다 작음",)
    )

In [25]:
client = ClientClass()
client.descriptor = 42
print("client.descriptor: ", client.descriptor)

client.descriptor:  42


In [26]:
client.descriptor = -42

ValueError: -42 는 0보다 작음

In [27]:
client.descriptor = "invalid value"

ValueError: 'invalid value' 는 숫자가 아님

위처럼 프로퍼티 자리에 놓일 수 있는 것은 디스크립터로 추상화할 수 있으며 여러번 재사용이 가능함  
위의 예에서는 &#95;&#95;set&#95;&#95;() 메서드가 @property.setter가 하던 일을 대신함